<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация-текста" data-toc-modified-id="Лемматизация-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация текста</a></span></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Векторизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li></ul></li><li><span><a href="#Проверка-LogisticRegression-на-тестовой-выборке" data-toc-modified-id="Проверка-LogisticRegression-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка LogisticRegression на тестовой выборке</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [32]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_csv('toxic_comments.csv')
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [8]:
data.duplicated().sum()

0

In [9]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [10]:
balance = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
balance 

8.834884437596301

**Вывод:**

Пропусков и дубликатов в данных не наблюдается. Баланс классов нарушен, негативных текстов примерно в девять раз меньше чем положительных

### Лемматизация текста

In [11]:
def clear_text(text):
    text = text.lower()
    clear_text = re.sub(r'[^a-zA-Z ]', ' ', text) 
    clear_text = clear_text.split()
    return " ".join(clear_text)

In [12]:
data['clear_text'] = data['text'].apply(lambda x: clear_text(x)) 
data = data.drop(['text'], axis=1)
data.head()

,toxic,clear_text
0,0,explanation why the edits made under my userna...
1,0,d aww he matches this background colour i m se...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestions on impr...
4,0,you sir are my hero any chance you remember wh...


In [13]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
  
lemmatizer = WordNetLemmatizer()

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
    
def my_lemmatizer(text):  
    # токенизация и определение части языка
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))  
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #лемма если нет тэга
            lemmatized_sentence.append(lemmatizer.lemmatize(word))
        else:        
            #лемма если тэг есть
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)

    return lemmatized_sentence

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hozin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hozin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hozin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [14]:
data['lemmas_text'] = data['clear_text'].apply(lambda x: my_lemmatizer(x))   
data.head()

,toxic,clear_text,lemmas_text
0,0,explanation why the edits made under my userna...,explanation why the edits make under my userna...
1,0,d aww he matches this background colour i m se...,d aww he match this background colour i m seem...
2,0,hey man i m really not trying to edit war it s...,hey man i m really not try to edit war it s ju...
3,0,more i can t make any real suggestions on impr...,more i can t make any real suggestion on impro...
4,0,you sir are my hero any chance you remember wh...,you sir be my hero any chance you remember wha...


### Векторизация

In [15]:
corpus = data['lemmas_text'].values.astype('U')

In [16]:
nltk.download('stopwords')
swords = stopwords.words('english')
stopwords_set = set(swords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hozin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords_set, ngram_range=(1, 1))

## Обучение

In [18]:
#обозначим признаки, разделим выборки на обучающую и тестовую
features = corpus
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)

### LogisticRegression

In [29]:
LR_model = LogisticRegression(class_weight = 'balanced', solver ='lbfgs', random_state=12345) 
pipe = Pipeline(steps=[('tf_idf', count_tf_idf), ('logistic', LR_model)])

parameters = {'logistic__C': np.linspace(0.0001, 10, 7)
             }

grid_search = GridSearchCV(pipe, parameters, scoring='f1', cv=3)
grid_search.fit(features_train, target_train)

print('F1: ', grid_search.best_score_)
print('Параметры : ', grid_search.best_params_)

F1:  0.7549588007298853
Параметры :  {'logistic__C': 10.0}


У Логистической регресии F1-мера равна 0.7549588007298853

### RandomForestClassifier

In [20]:
RFClf_model = RandomForestClassifier(class_weight = 'balanced', random_state=12345)
pipe = Pipeline(steps=[('tf_idf', count_tf_idf), ('RFClf', RFClf_model)])

parameters = {'RFClf__max_depth': np.linspace(10, 20, 16),
             'RFClf__n_estimators': range(40, 60, 80)}

grid_search = GridSearchCV(pipe, parameters, scoring= 'f1', cv= 3)
grid_search.fit(features_train, target_train)

print('F1: ', grid_search.best_score_)

F1:  0.38865234189028774


Метрика качества F1 равна 0.38865234189028774, что тоже ниже необходимого результата

### DecisionTreeClassifier

In [21]:
DTClf_model = DecisionTreeClassifier(class_weight = 'balanced', random_state=12345)
pipe = Pipeline(steps=[('tf_idf', count_tf_idf), ('DTClf', DTClf_model)])

parameters = {'DTClf__max_depth': np.linspace(10, 20, 16)}

grid_search = GridSearchCV(pipe, parameters, scoring= 'f1', cv= 3)
grid_search.fit(features_train, target_train)

print('F1: ', grid_search.best_score_)

F1:  0.5973504653466848


Метрика качества F1 равна  0.5973504653466848

## Проверка LogisticRegression на тестовой выборке

In [30]:
count_tf_idf.fit(features_train)

tf_idf_features_train = count_tf_idf.transform(features_train)
tf_idf_features_test = count_tf_idf.transform(features_test)

In [34]:
model = LogisticRegression(C=10.0, class_weight = 'balanced', solver ='lbfgs', random_state=12345) 

model.fit(tf_idf_features_train, target_train)
predictions = model.predict(tf_idf_features_test)

print('F1 на тестовой выборке у LogisticRegression: {:.2f}'.format(f1_score(target_test, predictions)))

F1 на тестовой выборке у LogisticRegression: 0.76


## Вывод

В ходе работы было выполнено следующее:

- Изучены и подготовлены данные.
- Сделана лемматизация и векторизация текста.
- Данные поделены на обучающую и тестовую выборки.
- Выбраны и обучены модели.

В итоге лучшей моделью стала LogisticRegression с метрикой качества F1 равной 0.76 на тестовых данных.